# SpinShots : Dynamic Image Rotation & GIF Maker

1.	이미지 불러오기: Pillow(PIL)를 이용해 이미지를 RGBA 모드로 불러옵니다.
2.	회전 및 중앙 정렬: 특정 각도(angle)로 이미지를 회전시킨 뒤, 주어진 캔버스 크기 내에서 중앙에 맞춰 붙여 넣습니다.
3.	애니메이션 프레임 생성: 여러 각도로 회전된 이미지를 프레임 리스트로 생성합니다.
4.	GIF 저장: 생성된 프레임 리스트를 이용해 GIF 파일로 저장합니다.

## 1. 라이브러리 임포트

Pillow(PIL) 라이브러리를 사용하기 위해 임포트합니다.

In [1]:
from PIL import Image

## 2. 입력(또는 설정) 값 정의

사용할 이미지 파일 이름, 캔버스 크기, 회전 각도 범위 등과 같은 설정값을 셀에서 정의합니다.

In [2]:
# 이미지 파일 이름(확장자 제외)
image_file_name = "IMG_6006"

# GIF로 만들고자 하는 캔버스 크기
canvas_size = (550, 550)

# 회전 각도 스텝(예: 0도~360도까지 몇 도 간격으로 회전할지)
angle_step = 60

# GIF 프레임 간 지속 시간(밀리초)
duration = 100

# 루프 횟수(0이면 무한 반복)
loop = 0

# 이미지 경로 및 GIF 출력 경로
image_path = f"assets/{image_file_name}.PNG"
output_gif_path = f"assets/{image_file_name}.gif"

print(f"이미지 경로: {image_path}")
print(f"GIF 출력 경로: {output_gif_path}")

이미지 경로: assets/IMG_6006.PNG
GIF 출력 경로: assets/IMG_6006.gif


## 3. 이미지 불러오기

이미지가 RGBA 모드로 열려 있는지 확인합니다. RGBA 모드로 변환해야 투명한 배경을 활용할 수 있습니다.

In [3]:
original_image = Image.open(image_path).convert("RGBA")
print(f"불러온 이미지 크기: {original_image.size}")

불러온 이미지 크기: (840, 675)


## 4. 회전 이미지를 캔버스 중앙에 배치하는 함수 정의

- image.rotate(angle, ...)로 이미지를 회전합니다.
- 회전된 이미지를 새 투명 캔버스(Image.new("RGBA", size, (255, 255, 255, 0)))에 중앙 정렬하여 붙여 넣습니다.

In [4]:
def create_rotated_frame(image: Image, rotate_angle: int, size: tuple[int, int]) -> Image:
    """
    주어진 image를 angle만큼 회전시키고,
    RGBA 모드의 투명 배경 캔버스에 중앙 정렬하여 합성한 후 반환합니다.

    :param image: 회전할 원본 이미지 (PIL Image 객체)
    :param rotate_angle: 회전 각도 (정수)
    :param size: 캔버스 크기 (튜플)
    :return: 중앙 정렬된 회전 이미지를 담은 RGBA 모드의 PIL Image 객체
    """
    # 이미지 회전
    rotated_image = image.rotate(rotate_angle, resample=Image.BICUBIC, expand=True)

    # 투명한 캔버스 생성
    canvas = Image.new("RGBA", size, (255, 255, 255, 0))

    # 중앙 정렬을 위한 x, y 좌표 오프셋 계산
    x_offset = (size[0] - rotated_image.width) // 2
    y_offset = (size[1] - rotated_image.height) // 2

    # 투명 캔버스에 붙여넣기
    canvas.paste(rotated_image, (x_offset, y_offset), rotated_image)

    return canvas

## 5. GIF 프레임 생성

- 0도부터 360도 직전까지 angle_step 간격으로 회전 각도를 지정해 순회합니다.
- create_rotated_frame() 함수를 통해 프레임을 생성하고 리스트에 담습니다.

In [5]:
frames = []
for angle in range(0, 360, angle_step):
    frame = create_rotated_frame(original_image, angle, canvas_size)
    frames.append(frame)

print(f"생성된 프레임 수: {len(frames)}")

생성된 프레임 수: 6


## 6. 프레임을 GIF로 저장하기

- 첫 번째 프레임을 기준으로 save_all=True, append_images=frames[1:] 옵션을 사용해 GIF 형태로 저장합니다.
- duration과 loop 옵션을 사용해 이미지 전환 속도와 재생 횟수를 설정합니다.
- disposal=2 옵션을 통해 프레임이 전환될 때마다 이전 프레임의 정보를 지우고 새 프레임으로 덮어쓰도록 설정합니다.

In [6]:
frames[0].save(
    output_gif_path,
    save_all=True,
    append_images=frames[1:],
    duration=duration,
    loop=loop,
    disposal=2
)

print("GIF 생성이 완료되었습니다!")
print(f"저장 경로: {output_gif_path}")

GIF 생성이 완료되었습니다!
저장 경로: assets/IMG_6006.gif
